In [20]:
import pandas as pd
from pandas_profiling import ProfileReport
import datetime

In [3]:
financial = pd.read_csv("../dataset/재무데이터.txt", sep = '\t', encoding='cp949')
non_financial = pd.read_csv('../dataset/merge.csv')
active_inactive = pd.read_excel('../dataset/액티브 중소법인 재무보유.xlsx', sheet_name='휴폐업이력')
inactive_inactive = pd.read_excel('../dataset/휴폐업 중소법인 재무보유.xlsx', sheet_name='휴폐업이력')

In [52]:
isinactive = non_financial[['사업자등록번호', '휴폐업 유무']] # 현재 휴폐업 상태 / Y: 휴폐업, N: 액티브
isinactive

,사업자등록번호,휴폐업 유무
0,1138111739,N
1,2078132193,N
2,1318167565,N
3,5048128251,N
4,1098164260,N
...,...,...
234367,4808501629,Y
234368,8018501578,Y
234369,4118523419,Y
234370,1738102533,Y


## 회계결산 횟수

In [55]:
financial.head(10)
finance = financial[['사업자번호', '결산년월']]
finance['결산년월'] = pd.to_datetime(df1['결산년월'], format='%Y%m%d')
finance

/var/folders/8t/p9vrbnhn7xj3s_2zmct430yw0000gn/T/ipykernel_80519/3491751163.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finance['결산년월'] = pd.to_datetime(df1['결산년월'], format='%Y%m%d')


,사업자번호,결산년월
0,1018100340,2018-12-31
1,1018100340,2019-12-31
2,1018100340,2020-12-31
3,1018100340,2021-12-31
4,1018100772,2018-12-31
...,...,...
109137,8998800427,2021-12-31
109138,8998800785,2018-12-31
109139,8998800785,2019-12-31
109140,8998800785,2020-12-31


In [60]:
cnt_finance =finance.groupby(['사업자번호']).count()
cnt_finance = cnt_finance.rename(columns = {'결산년월':'CNT_FINACNE'})
cnt_finance

,CNT_FINACNE
사업자번호,
1018100340,4
1018100772,4
1018101126,4
1018101242,4
1018103819,4
...,...
8998700283,2
8998701263,3
8998800150,4


## 휴폐업 횟수 피처

In [37]:
cnt_inactive = inactive.groupby(['BIZ_NO']).count()
cnt_inactive

,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE
BIZ_NO,,,,
1018135422,1,1,1,1
1018154206,1,1,1,1
1018161162,2,2,2,0
1018163684,1,1,1,1
1018163758,4,4,4,2
...,...,...,...,...
8998100409,4,4,4,1
8998600312,1,1,1,1
8998700089,2,2,2,0


In [51]:
many_inactive = cnt_inactive[cnt_inactive['END_DATE'] > 1]['END_DATE']
many_inactive = many_inactive.rename('CNT_INACTIVE')
many_inactive

BIZ_NO
1018161162    2
1018163758    4
1018180233    2
1018188967    4
1018189006    2
             ..
8898701043    2
8918100104    2
8938100809    2
8998100409    4
8998700089    2
Name: CNT_INACTIVE, Length: 2131, dtype: int64

## 재무 결산년월에 휴폐업 상태였는지?
 - 휴폐업 상태인데 회계결산?
 - 이전 결산일을 반영?

In [73]:
inactive = pd.concat([active_inactive, inactive_inactive]).reset_index(drop=True)
inactive['END_DATE'] = inactive['END_DATE'].replace(99991231, 20991231)
inactive['STRT_DATE'] = pd.to_datetime(inactive['STRT_DATE'], format='%Y%m%d')
inactive['END_DATE'] = pd.to_datetime(inactive['END_DATE'], format='%Y%m%d')

inactive

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE
0,5088115592,2099-12-31,2015-02-09,5,NaN
1,5088115592,2015-02-08,2020-07-31,2,NaN
2,1138127100,2099-12-31,2012-11-05,5,NaN
3,1138127100,2012-11-04,2020-07-31,2,NaN
4,1138127100,2012-10-04,2020-07-31,5,NaN
...,...,...,...,...,...
11662,4808501629,2099-12-31,2021-12-31,3,20211231.0
11663,8018501578,2099-12-31,2021-11-26,3,20211126.0
11664,4118523419,2099-12-31,2022-06-30,3,20220630.0
11665,1738102533,2099-12-31,2022-07-31,3,20220731.0


In [115]:
def isinactive(biz_no, set_day):
    if len(inactive[(inactive['BIZ_NO'] == biz_no) & (inactive['STRT_DATE'] <= set_day) & (set_day <= inactive['END_DATE'])]) >= 1:
        return True
    else:
        return False

In [118]:
finance['IS_INACTIVE_AT'] = finance.apply(lambda x: isinactive(x['사업자번호'], x['결산년월']), axis=1)
finance

/var/folders/8t/p9vrbnhn7xj3s_2zmct430yw0000gn/T/ipykernel_80519/1856049249.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finance['IS_INACTIVE_AT'] = finance.apply(lambda x: isinactive(x['사업자번호'], x['결산년월']), axis=1)


,사업자번호,결산년월,IS_INACTIVE_AT
0,1018100340,2018-12-31,False
1,1018100340,2019-12-31,False
2,1018100340,2020-12-31,False
3,1018100340,2021-12-31,False
4,1018100772,2018-12-31,False
...,...,...,...
109137,8998800427,2021-12-31,False
109138,8998800785,2018-12-31,False
109139,8998800785,2019-12-31,False
109140,8998800785,2020-12-31,False


In [121]:
print(finance['IS_INACTIVE_AT'].value_counts())

finance[finance['IS_INACTIVE_AT'] == True]

False    108341
True        801
Name: IS_INACTIVE_AT, dtype: int64


,사업자번호,결산년월,IS_INACTIVE_AT
263,1018161162,2018-12-31,True
264,1018161162,2019-12-31,True
265,1018161162,2020-12-31,True
266,1018161162,2021-12-31,True
385,1018189006,2020-12-31,True
...,...,...,...
103959,6948600212,2020-12-31,True
104127,7028100143,2021-12-31,True
104413,7118801975,2021-12-31,True
106408,7958600567,2021-12-31,True
